# Finetune Models

In [3]:
import json


dataset = json.loads(open("data/paths.json").read())


print(dataset["a3"])

['program -> return_statement -> object_creation_expression -> generic_type -> scoped_type_identifier -> AbstractMap', 'program -> return_statement -> object_creation_expression -> generic_type -> scoped_type_identifier -> SimpleImmutableEntry', 'program -> return_statement -> object_creation_expression -> generic_type -> type_arguments', 'program -> return_statement -> object_creation_expression -> argument_list -> "a"', 'program -> return_statement -> object_creation_expression -> argument_list -> 3']


In [4]:
formatted_dataset = [
    {"method_name": method, "text": " | ".join(dataset[method])} for method in dataset
]


print(formatted_dataset[0])

{'method_name': 'a3', 'text': 'program -> return_statement -> object_creation_expression -> generic_type -> scoped_type_identifier -> AbstractMap | program -> return_statement -> object_creation_expression -> generic_type -> scoped_type_identifier -> SimpleImmutableEntry | program -> return_statement -> object_creation_expression -> generic_type -> type_arguments | program -> return_statement -> object_creation_expression -> argument_list -> "a" | program -> return_statement -> object_creation_expression -> argument_list -> 3'}


In [22]:
from transformers import RobertaTokenizer
from datasets import Dataset
from pandas import DataFrame

tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base-mlm")


def tokenize_function(method):
    model_inputs = tokenizer(
        method["text"],
        padding="max_length",
        truncation=True,
    )
    model_inputs["masked_methods"] = tokenizer(method["method_name"])["input_ids"]
    return model_inputs


tokenized_dataset = Dataset.from_pandas(DataFrame(formatted_dataset)).map(
    tokenize_function, batched=True
)

print(tokenized_dataset[0])
print(len(tokenized_dataset))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

c:\Users\jaspe\Documents\Group02\.venv\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jaspe\.cache\huggingface\hub\models--microsoft--codebert-base-mlm. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/504 [00:00<?, ?B/s]

Map:   0%|          | 0/3620 [00:00<?, ? examples/s]

{'method_name': 'a3', 'text': 'program -> return_statement -> object_creation_expression -> generic_type -> scoped_type_identifier -> AbstractMap | program -> return_statement -> object_creation_expression -> generic_type -> scoped_type_identifier -> SimpleImmutableEntry | program -> return_statement -> object_creation_expression -> generic_type -> type_arguments | program -> return_statement -> object_creation_expression -> argument_list -> "a" | program -> return_statement -> object_creation_expression -> argument_list -> 3', 'input_ids': [0, 28644, 43839, 671, 1215, 29963, 43839, 7626, 1215, 39025, 1215, 42819, 43839, 14569, 1215, 12528, 43839, 2850, 15911, 1215, 12528, 1215, 8009, 24072, 43839, 43649, 41151, 1721, 586, 43839, 671, 1215, 29963, 43839, 7626, 1215, 39025, 1215, 42819, 43839, 14569, 1215, 12528, 43839, 2850, 15911, 1215, 12528, 1215, 8009, 24072, 43839, 21375, 37380, 41280, 46640, 1721, 586, 43839, 671, 1215, 29963, 43839, 7626, 1215, 39025, 1215, 42819, 43839, 14569, 

## Cool example of tokens to words

In [23]:
for word in tokenized_dataset[0]["input_ids"]:

    print(f"{word} {tokenizer.decode(word)}")

 some
0 <s>
28644 program
43839  ->
671  return
1215 _
29963 statement
43839  ->
7626  object
1215 _
39025 creation
1215 _
42819 expression
43839  ->
14569  generic
1215 _
12528 type
43839  ->
2850  sc
15911 oped
1215 _
12528 type
1215 _
8009 ident
24072 ifier
43839  ->
43649  Abstract
41151 Map
1721  |
586  program
43839  ->
671  return
1215 _
29963 statement
43839  ->
7626  object
1215 _
39025 creation
1215 _
42819 expression
43839  ->
14569  generic
1215 _
12528 type
43839  ->
2850  sc
15911 oped
1215 _
12528 type
1215 _
8009 ident
24072 ifier
43839  ->
21375  Simple
37380 Imm
41280 utable
46640 Entry
1721  |
586  program
43839  ->
671  return
1215 _
29963 statement
43839  ->
7626  object
1215 _
39025 creation
1215 _
42819 expression
43839  ->
14569  generic
1215 _
12528 type
43839  ->
1907  type
1215 _
5384 arg
30179 uments
1721  |
586  program
43839  ->
671  return
1215 _
29963 statement
43839  ->
7626  object
1215 _
39025 creation
1215 _
42819 expression
43839  ->
4795  argument


In [34]:
# Give the tokenized datset "train and test" labels
split_tokenized_dataset = tokenized_dataset.train_test_split(
    test_size=0.2, train_size=0.8, shuffle=True, seed=42
)
print(split_tokenized_dataset["train"][0])
print(split_tokenized_dataset["test"][0])

print(split_tokenized_dataset["train"].keys())

{'method_name': 'createSourceAddrPanel', 'text': 'program -> local_variable_declaration -> modifiers | program -> local_variable_declaration -> JPanel | program -> local_variable_declaration -> variable_declarator -> sourceAddrPanel | program -> local_variable_declaration -> variable_declarator -> object_creation_expression -> HorizontalPanel | program -> local_variable_declaration -> variable_declarator -> object_creation_expression -> argument_list | program -> expression_statement -> method_invocation -> sourceAddrPanel | program -> expression_statement -> method_invocation -> setBorder | program -> expression_statement -> method_invocation -> argument_list -> method_invocation -> BorderFactory | program -> expression_statement -> method_invocation -> argument_list -> method_invocation -> createTitledBorder | program -> expression_statement -> method_invocation -> argument_list -> method_invocation -> argument_list -> method_invocation -> JMeterUtils | program -> expression_statemen

AttributeError: 'Dataset' object has no attribute 'keys'

In [11]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)

    predictions_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions
    ]

    result["gen_len"] = np.mean(predictions_lens)

    return {k: round(v, 4) for k, v, in result.items()}

In [31]:
from transformers import BertForMaskedLM, BertConfig, AdamW


config = BertConfig.from_pretrained("microsoft/codebert-base-mlm")


config.is_decoder = True


# model = AutoModel.from_pretrained(

#   "microsoft/unixcoder-base",

#   config=config,

#


training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
)


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=split_tokenized_dataset["train"],
    eval_dataset=split_tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


<built-in method keys of DatasetDict object at 0x000001E20248C0B0>


ValueError: FP16 Mixed precision training with AMP or APEX (`--fp16`) and FP16 half precision evaluation (`--fp16_full_eval`) can only be used on CUDA or NPU devices or certain XPU devices (with IPEX).

In [28]:
from transformers import RobertaForMaskedLM, pipeline

model = RobertaForMaskedLM.from_pretrained("microsoft/codebert-base-mlm")

code_example = "public int <mask><mask>(a,b){return a * b;}"
fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer,
)

for itw in fill_mask(code_example):
    print(itw)

Some weights of the model checkpoint at microsoft/codebert-base-mlm were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.28434231877326965, 'token': 14445, 'token_str': ' div', 'sequence': '<s>public int div<mask>(a,b){return a * b;}</s>'}, {'score': 0.03896702826023102, 'token': 17895, 'token_str': ' sq', 'sequence': '<s>public int sq<mask>(a,b){return a * b;}</s>'}, {'score': 0.03892657905817032, 'token': 3023, 'token_str': ' x', 'sequence': '<s>public int x<mask>(a,b){return a * b;}</s>'}, {'score': 0.03314271569252014, 'token': 740, 'token_str': ' c', 'sequence': '<s>public int c<mask>(a,b){return a * b;}</s>'}, {'score': 0.02497895620763302, 'token': 6979, 'token_str': ' int', 'sequence': '<s>public int int<mask>(a,b){return a * b;}</s>'}]
[{'score': 0.06596952676773071, 'token': 2765, 'token_str': 'By', 'sequence': '<s>public int<mask>By(a,b){return a * b;}</s>'}, {'score': 0.04894856736063957, 'token': 118, 'token_str': 'i', 'sequence': '<s>public int<mask>i(a,b){return a * b;}</s>'}, {'score': 0.045959293842315674, 'token': 10643, 'token_str': 'Of', 'sequence': '<s>public int<mask>Of